In [1]:
from builtins import Exception
import urllib3
from bs4 import BeautifulSoup
from datetime import datetime
from requests import get
import mysql.connector

# Deklarasi Connection Database

In [2]:
mydb = mysql.connector.connect(
    host="localhost",
    user="root",
    password="",
    database="db-berita-mca"  
) 

# Fungsi-Fungsi Pada Database

Kernel kebawah tersebut merupakan fungsi-fungsi yang dibuat untuk sebuah action pada database

[simpanDatabase] Menyimpan Data Kedalam Database

In [3]:
def simpanDatabase(mydb,data):
    mycursor = mydb.cursor()
    sql = "INSERT INTO tb_berita (media,judul_berita,label,penulis,release_date,url,content) VALUES (%s,%s,%s,%s,%s,%s,%s)"
    mycursor.execute(sql, data)
    mydb.commit()
    return 

[ValidasiDataDatabase] Mengecek kedalam database apakah data tersebut sudah pernah di simpan atau belum

In [4]:
def validasiDataDatabase(mydb,dataBerita):
    mycursor = mydb.cursor()
    sql = "SELECT * FROM tb_berita WHERE url = %s"
    dataUrl = dataBerita
    url = (dataBerita,)
    mycursor.execute(sql, url)
    myresult = mycursor.fetchall()
    return myresult

# Semua Halaman Scraping

Dibawah ini adalah source yang digunakan dalam mengambil berapa panjang dari halaman website CBNC Indonesia

In [5]:
def getAllPageWebsite(url,keyword):
    if(keyword == ""):
        urlGet = (url+"/indeks")
    else:
        urlGet = (url+"/search?query="+keyword)
        
    try:
        soup = BeautifulSoup(get(urlGet).text, 'lxml')
        getAllPage= soup.find('div','paging text_center gtm_paging')
        page_link = getAllPage.select('a')
    except Exception as e:
        print(e)
    
    try:
        for page in page_link:
            if(page.text != ""):
                lastPage = page.text
        return lastPage
    except Exception as e:
        print("Tidak Terdapat Data pada Pencarian anda tersebut")

# Get Data URL Detail Berita

Fungsi dibawah ini ngambil semua url berdasarkan indeks page yang tersedia pada website CBNC

In [6]:
def getUrlInPage(url,keyword,page):
    urlDetailData = []
    for page in range(page): 
        if(keyword == ""):
            urlGet = (url+"/indeks/{}".format(page+1))
        else:
            urlGet = (url+"/search?query="+keyword+"&p={}".format(page+1))
            
        indeksPage = get(urlGet)
        soup = BeautifulSoup(indeksPage.text, 'html.parser')
        contents = soup.find_all('article')
        for content in contents:
            try:
                news_url = content.find('a', href=True).get('href')
                urlDetailData.append(news_url)
            except Exception as e:
                continue
    return urlDetailData

# Scraping Data Detail News

Scraping data detail berdasarkan url yang diambil perhalaman diatas
nah dibawah inin ni isi mau aku buatin buat ngecek gak double insert data aja dia deh wait


In [7]:
def scrapingDataDetail(urlDetailData,mydb):
    for dataBerita in urlDetailData:
            
        myresult = validasiDataDatabase(mydb,dataBerita)
        
        if(len(myresult)==0):
            try:
                web_data = get(dataBerita)
                soup = BeautifulSoup(web_data.text, 'html.parser')
                header = soup.find('div', class_='jdl')
                media = "CBNC Indonesia"
                title = header.find('h1').text
                author_class = header.find('div', class_='author').text.split(' ')
                label = author_class[0]
                author = ' '.join(author_class[2:])
                release_date = header.find('div', class_='date').text
                detail_text_class = soup.find('div', class_='detail_text')
                texts = detail_text_class.find_all('p')
                news_content = ' '.join([text.text for text in texts])
        #         news_content_data = {
        #             'media': "CBNC Indonesia",
        #             'title': title,
        #             'label': label,
        #             'author': author,
        #             'release_date': release_date,
        #             'url':dataBerita,
        #             'content': news_content
        #         }

                data = (media,title,label,author,release_date,dataBerita,news_content)
                simpanDatabase(mydb,data)
            except Exception as e:
                continue    
        else:
            continue

# Fungsi Utama Engine

In [8]:
url = "https://www.cnbcindonesia.com"
search = ""
allPage = getAllPageWebsite(url,search)

# parameter(url,keyword,page)
urlDetail = getUrlInPage(url,search,1)

scrapingDataDetail(urlDetail,mydb)

Deklarasi Penjadwalan Engine berjalan

In [9]:
import schedule
import time

schedule.every().day.at("23:00").do(main,data)

while True:
    schedule.run_pending()
    time.sleep(60) # wait one minute